Day 18 of Advent of Code 2021

February 4, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(linewidth=250)
import json


In [2]:


class Snail():
    """
    Using ridiculous naming for now in that the class is Snail and inside the class
    the main item is called hw for homework.  
    
    The main item, hw, is a string.
    """
    
    hw = ''
    left = '['
    right = ']'
    edepth = 5
    num_max = 9
    left_score_multiplier = 3
    right_score_multiplier = 2
    
    
    def __init__(self,s):
        
        self.hw = s
        
    def __repr__(self):
        
        return self.hw
    
    def add_string(self,mhw):
        """
        Input:
            mhw    type Class Snail
        
        Returns the addition string when adding mhw *on the right*
        """
        return '[' + self.hw + ',' + mhw.hw + ']'
    
    def depth(self):
        """
        Determines how deep the brackets go
        """
        um = 0
        track = 0
        for s in self.hw:
            if s == self.left:
                track = track + 1
                if track > um:
                    um = track
            elif s == self.right:
                track = track - 1
        return um
    
    def comma_split(self):
        return self.hw.split(',')
    
    def left_counts(self):
        return [x.count(self.left) for x in self.comma_split()]
        
    def right_counts(self):
        return [x.count(self.right) for x in self.comma_split()]
    
    def depth_array(self):
        """
        Returns an array of the open depth at the end of each comma separated term.
        """
        return list(np.cumsum(self.left_counts()) - np.cumsum(self.right_counts()))
    
    def extract_number(self, g):
        return int(''.join([x for x in g if x not in [self.left,self.right]]))

    def number_array(self):
        """
        Returns an array of the numbers in each of the comma separated terms
        """
        return [self.extract_number(x) for x in self.comma_split()]
    
    def replace_number(self,old_term,new_number):
        old_number = self.extract_number(old_term)
        return old_term.replace(str(old_number),str(new_number))
        
        
    
    def single_boom(self):
        """
        Finds the first pair that should explode in the self.hw string and then 
        returns the post explosion string.  If no strings can be exploded returns
        the original self.hw
        
        WARNING!!!! DO NOT USE THIS FUNCTION.  IT HAS A BUG THAT IS NOT GOING TO BE 
        FIXED.  USE boom INSTEAD
        """
        cs = self.comma_split()
        da = self.depth_array()
        if self.edepth in da:
            findx = da.index(self.edepth)
            alist = cs[:findx]
            blist = cs[findx]
            clist = cs[findx+1:]
        
            
            nlist = []            
            if len(alist) == 0:
                nlist.append(self.replace_number(blist,0)[1:])
            else:
                #Put all of the things that are not the last item into nlist
                nlist.extend(alist[:-1])
                #Find the last item and modify by adding in the going away number and 
                #add it to the list
                last_item = alist[-1]
                first_num = self.extract_number(blist)
                last_num = self.extract_number(last_item)
                nlist.append(self.replace_number(last_item,first_num + last_num))
            
            #Next up we must deal with the other side
            assert(len(clist) > 0)
            #####WARNING BUG IS HERE---WE ASSUMED THAT THERE WERE ALWAYS MORE RIGHT BRACKETS
            ######       THAN LEFT BRACKETS.  OOOOPPPSSS....
            second_num = self.extract_number(clist[0])
            if len(alist) > 0:
                nlist.append(self.replace_number(clist[0],0)[:-1])
            if len(clist) > 1:
                third_num = self.extract_number(clist[1])
                nlist.append(self.replace_number(clist[1],second_num + third_num))
                nlist.extend(clist[2:])
                
            return ','.join(nlist)
        else:
            return self.hw
 
    def boom(self):
        """
        Finds the first pair that should explode in the self.hw string and then 
        returns the post explosion string.  If no strings can be exploded returns
        the original self.hw
        
        NOTE:  This is the updated version that hopefully works correctly.
        """
        cs = self.comma_split()
        da = self.depth_array()
        if self.edepth in da:
            findx = da.index(self.edepth)
            nums = self.number_array()
            #Now we add the correct numbers into the appropriate terms.
            if findx> 0:
                cs[findx-1] = self.replace_number(cs[findx-1],nums[findx - 1] + nums[findx])
            if len(cs) - findx > 2:
                cs[findx+2] = self.replace_number(cs[findx+2],nums[findx + 2] + nums[findx+1])
            
            #Next we combine the two middle terms into a single appropriate term.
            lc = self.left_counts()[findx]
            rc = self.right_counts()[findx + 1]
            assert(lc != rc)
            if lc < rc:
                nt = '0'+self.right*(rc-1)
            else:
                nt = self.left*(lc-1) + '0'
        
            
            return ','.join(cs[:findx] + [nt] + cs[findx+2:])
        else:
            return self.hw
        




    def explode(self, verbose = False):
        """
        IMPORTANT!  
             THIS FUNCTION MODIFIES THE hw ELEMENT OF THE CLASS.  So it will change 
             the values.
        """
        
        while self.hw != self.boom():
            if verbose:
                print(self.hw)
            self.hw = self.boom()
    
    def get_split_indices(self):
        nums = np.array(self.number_array())
        return list(np.where(nums > self.num_max)[0])

    
    def single_split(self):
        """
        Finds the first instance where the number is > 9 and returns the string where 
        this instance has been split.  If no such instance exists, it returns self.hw.
        """
        party = self.get_split_indices()
        if len(party) >0:
            sindx = party[0]
            cs = self.comma_split()
            surgery_term = cs[sindx]
            surgery_number = self.extract_number(surgery_term)
            first = int(surgery_number/2)
            second = surgery_number - first
            ###NOTE:  self.replace_number wasn't originally intended to be used for strings.
            ###  but it just happens to work anyway.  Potential source for future bugs here.
            surgery_term = self.replace_number(surgery_term,'[{},{}]'.format(first,second))
            cs[sindx] = surgery_term
            return ','.join(cs)
        else:
            return self.hw
        
        
    def reduce(self, verbose= False):
        """
        IMPORTANT!  
             THIS FUNCTION MODIFIES THE hw ELEMENT OF THE CLASS.  So it will change 
             the values.
        """
        self.explode()
        old = self.hw
        while old != self.single_split():
            if verbose:
                print('Before  split:  ', self.hw)
            self.hw = self.single_split()
            if verbose:
                print('After   split:  ', self.hw)
                
            self.explode()
            if verbose:
                print('After explode:  ', self.hw)
                print()

            old = self.hw

    def add(self,other_snail,verbose = False):
        """
        IMPORTANT!  
             THIS FUNCTION MODIFIES THE hw ELEMENT OF THE CLASS.  So it will change 
             the values.
             
        Furthermore:  other_snail is assumed to also be an instance of Snail.
        
        Furthermore:  Order is very important to this and other_snail is added on the 
        right!
        """
        if verbose:
            print("Reducing self")
        self.reduce(verbose = verbose)
        if verbose:
            print("Reducing other_snail")
        other_snail.reduce(verbose = verbose)
        self.hw = self.add_string(other_snail)
        if verbose:
            print('Added them together')
            print(self.hw)
            print("Reducing result")
        self.reduce(verbose = verbose)
   

#####
#
# Functions for computing the magnitude of a snail.
#
####
            
"""
Is this the crappiest implementation of a scoring function?  Possibly yes.  

Do I care at this point?  Definitely not.

Certainly not enough to figure out why the recusion is only partially working.  Am 
applying a 'bandage' and moving on to *any* other thing I could be doing.

So the magnitude scoring values are nasty little bits of hardcoding and I have to 
call the recursion multiple times.  And it assumes that the input is a Snail class
without checking.  The list goes on.

"""

def grover(a, verbose = False):
    if verbose:
        print(a)
    assert(len(a) == 2)
    if type(a[0]) is int:
        if type(a[1]) is int:
            rval = 3*a[0] + 2*a[1]
        else:
            rval = [a[0],grover(a[1])]
    else:
        if type(a[1]) is int:
            rval = [grover(a[0]),a[1]]
        else:
            rval = [grover(a[0]),grover(a[1])]
    if verbose:
        print(rval)
    return rval
                    

def magnitude(snail):
    """
    This could run forever.  You have been warned.
    """
    a = json.loads(snail.hw)
    b = grover(a)
    while type(b) is not int:
        b = grover(b)
    return b
    

        
###All of these are here because lots of test cases are necessary.    
    
#Basic examples
t = Snail('[2,3]')
u = Snail('[[8,4],3]')
#Examples pre and post where explosions are necessary
k = Snail('[[[[[9,8],1],2],3],4]')
pk = Snail('[[[[0,9],2],3],4]')

tp = Snail('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')
ptp = Snail('[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]')
pptp = Snail('[[3,[2,[8,0]]],[9,[5,[7,0]]]]')

#Example that looks complicated but is actually in 'reduced form'
tp2 = Snail('[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]')

#Split mid examples
j = Snail('[[[[0,7],4],[15,[0,13]]],[1,1]]')
sj = Snail('[[[[0,7],4],[[7,8],[0,13]]],[1,1]]')
ssj = Snail('[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]')
fssj = Snail('[[[[0,7],4],[[7,8],[6,0]]],[8,1]]')

In [3]:
with open("18s.txt") as f:
    slines = f.readlines()
slines = [x.strip() for x in slines]
with open("18.txt") as f:
    contents = f.readlines()
contents = [x.strip() for x in contents]

In [4]:
slines

['[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
 '[[[5,[2,8]],4],[5,[[9,9],0]]]',
 '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
 '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
 '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
 '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
 '[[[[5,4],[7,7]],8],[[8,3],8]]',
 '[[9,3],[[9,9],[6,[4,9]]]]',
 '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
 '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]']

In [5]:
sample_snail = Snail(slines[0])
sample_snail

[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]

In [6]:
for s in slines[1:]:
    sample_snail.add(Snail(s))
    print(sample_snail)
magnitude(sample_snail)

[[[[7,0],[7,8]],[[7,9],[0,6]]],[[[7,0],[6,6]],[[7,7],[0,9]]]]
[[[[7,7],[7,7]],[[7,0],[7,7]]],[[[7,7],[6,7]],[[7,7],[8,9]]]]
[[[[6,6],[6,6]],[[7,7],[7,7]]],[[[7,0],[7,7]],[[7,8],[8,8]]]]
[[[[6,6],[7,7]],[[7,7],[8,8]]],[[[8,8],[0,8]],[[8,9],[9,9]]]]
[[[[6,6],[7,7]],[[7,7],[7,0]]],[[[7,7],[8,8]],[[8,8],[8,9]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[0,7],[8,8]],[[8,8],[8,9]]]]
[[[[7,7],[7,7]],[[7,7],[8,8]]],[[[8,8],[0,8]],[[8,9],[8,7]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[8,7],[8,7]],[[7,9],[5,0]]]]
[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]


4140

In [7]:
contents

['[[0,6],[[[4,0],[6,6]],[[2,2],9]]]',
 '[[9,[[1,6],[6,0]]],[[1,[0,8]],[[0,8],[9,8]]]]',
 '[[[0,[2,1]],3],[[[2,4],[1,2]],[7,5]]]',
 '[[[[8,3],[8,5]],[[7,8],[5,5]]],[9,2]]',
 '[[8,[1,9]],[[[9,9],[9,2]],1]]',
 '[[[[3,7],[2,1]],[0,9]],4]',
 '[[[[3,8],[6,0]],[0,7]],[[[6,3],[2,0]],9]]',
 '[[[9,[7,0]],[8,[9,6]]],[[5,6],4]]',
 '[[[[3,6],[3,6]],[0,2]],[[[8,3],9],[[3,4],8]]]',
 '[[7,[8,4]],1]',
 '[6,[[3,[5,6]],[0,6]]]',
 '[[[7,[4,7]],[[4,5],[4,3]]],[[5,5],[0,[4,2]]]]',
 '[[[0,[2,9]],[[2,4],[4,8]]],[[8,[9,5]],[[9,6],0]]]',
 '[[[[2,0],[9,7]],[[3,2],0]],[7,7]]',
 '[[5,[2,1]],[[3,[5,1]],[[8,5],[1,8]]]]',
 '[[[[9,7],6],[[7,8],7]],[[[6,8],9],[[9,5],7]]]',
 '[[4,2],[[[0,1],[7,2]],[[0,2],[5,5]]]]',
 '[[1,8],[[5,[7,9]],[[3,1],[7,1]]]]',
 '[[[4,[4,6]],6],5]',
 '[[[5,[3,6]],6],[[[8,0],[8,6]],[[3,3],[0,1]]]]',
 '[[4,[[2,6],[0,9]]],[[0,6],[4,2]]]',
 '[[[[9,4],[6,5]],7],[[[1,5],[0,9]],[4,[4,2]]]]',
 '[[7,[[6,5],8]],[[[5,6],0],[6,[3,5]]]]',
 '[[[5,[6,4]],[8,[0,4]]],[[3,[9,3]],4]]',
 '[[[[4,0],6],[6,[6,5]]],[[9

In [8]:
crazy_snail = Snail(contents[0])
print(crazy_snail)
for s in contents[1:]:
    crazy_snail.add(Snail(s))
    print(crazy_snail)
magnitude(crazy_snail)

[[0,6],[[[4,0],[6,6]],[[2,2],9]]]
[[[0,[5,5]],[[6,8],[5,7]]],[[[0,8],[7,6]],[[9,0],[8,9]]]]
[[[[6,6],[0,7]],[[7,7],[7,7]]],[[[6,7],[7,7]],[[5,0],[9,5]]]]
[[[[6,7],[7,7]],[[7,7],[0,7]]],[[[7,7],[7,7]],[[7,8],[8,8]]]]
[[[[6,6],[7,7]],[[7,7],[7,7]]],[[[0,7],[7,8]],[[8,8],[8,8]]]]
[[[[7,8],[7,8]],[[8,8],[8,8]]],[[[7,0],[8,7]],[9,[5,5]]]]
[[[[7,7],[7,0]],[[7,7],[7,7]]],[[[7,7],[7,7]],[[7,7],[7,8]]]]
[[[[7,7],[7,7]],[[7,7],[0,7]]],[[[7,7],[7,7]],[[7,8],[8,9]]]]
[[[[6,6],[6,6]],[[6,6],[0,7]]],[[[7,7],[7,7]],[[8,8],[8,7]]]]
[[[[7,7],[7,7]],[[7,7],[8,8]]],[[[8,0],[9,9]],[6,6]]]
[[[[6,7],[0,7]],[[7,7],[8,9]]],[[9,[5,5]],[[8,0],[6,6]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[7,7],[8,8]],[[0,9],[9,8]]]]
[[[[7,7],[7,7]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[8,8],[8,8]]]]
[[[[7,0],[7,8]],[[7,7],[8,8]]],[[[8,7],[6,7]],[[7,7],[6,6]]]]
[[[[7,0],[7,7]],[[7,7],[7,7]]],[[[8,7],[7,9]],[[8,9],[6,0]]]]
[[[[7,7],[7,7]],[[0,7],[7,7]]],[[[8,8],[8,8]],[[8,8],[8,9]]]]
[[[[0,7],[8,8]],[[8,8],[7,8]]],[[[8,7],[8,9]],[[8,2],[7,

4088

#### Part two

Which can be classified under 'could have been much worse'

In [9]:
def part_two(slist):
    max_score = 0
    s1 = ''
    s2 = ''
    for one in slist:
        for two in slist:
            if one != two:
                s = Snail(one)
                s.add(Snail(two))
                sc = magnitude(s)
                if sc > max_score:
                    max_score = sc
                    s1 = one
                    s2 = two
                s = Snail(two)
                s.add(Snail(one))
                sc = magnitude(s)
                if sc > max_score:
                    max_score = sc
                    s1 = two
                    s2 = one
    return max_score, s1, s2
                    

In [10]:
part_two(slines)

(3993,
 '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
 '[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]')

In [11]:
part_two(contents)

(4536,
 '[[[1,[7,7]],[[2,2],8]],[[[9,7],5],[4,3]]]',
 '[[[[8,2],0],[[9,6],[9,0]]],[[[6,2],[5,0]],9]]')

### Tests of the class functions

In [12]:
t.add_string(k)

'[[2,3],[[[[[9,8],1],2],3],4]]'

In [13]:
tp.depth()

5

In [14]:
tp2.depth()

4

In [15]:
tp2.comma_split()

['[[[0', '[4', '5]]', '[0', '0]]', '[[[4', '5]', '[2', '6]]', '[9', '5]]]']

In [16]:
tp.comma_split()

['[[3', '[2', '[1', '[7', '3]]]]', '[6', '[5', '[4', '[3', '2]]]]]']

In [17]:
tp.left_counts()

[2, 1, 1, 1, 0, 1, 1, 1, 1, 0]

In [18]:
tp.right_counts()

[0, 0, 0, 0, 4, 0, 0, 0, 0, 5]

In [19]:
tp.depth_array()

[2, 3, 4, 5, 1, 2, 3, 4, 5, 0]

In [20]:

print(k)
print(k.single_boom())
print(k.boom())
print(pk)
k.single_boom() == pk.hw

[[[[[9,8],1],2],3],4]
[[[[0,9],2],3],4]
[[[[0,9],2],3],4]
[[[[0,9],2],3],4]


True

In [21]:
print(tp)
print(tp.single_boom())
print(tp.boom())
print(ptp)
tp.single_boom() == ptp.hw


[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]


True

In [22]:
print(ptp)
print(ptp.single_boom())
print(ptp.boom())
print(pptp)
ptp.single_boom() == pptp.hw

[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]


True

In [23]:
print(pptp.depth_array())
pptp.depth()

[2, 3, 4, 1, 2, 3, 4, 0]


4

In [24]:
j = Snail('[[[[0,7],4],[15,[0,13]]],[1,1]]')
print(j)
print(sj)
print(ssj)
print(fssj)
print()
j.reduce(verbose = True)

[[[[0,7],4],[15,[0,13]]],[1,1]]
[[[[0,7],4],[[7,8],[0,13]]],[1,1]]
[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

Before  split:   [[[[0,7],4],[15,[0,13]]],[1,1]]
After   split:   [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
After explode:   [[[[0,7],4],[[7,8],[0,13]]],[1,1]]

Before  split:   [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
After   split:   [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
After explode:   [[[[0,7],4],[[7,8],[6,0]]],[8,1]]



In [25]:
big_example = ['[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]',
               '[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]',
               '[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]',
               '[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]',
               '[7,[5,[[3,8],[1,4]]]]',
               '[[2,[2,2]],[8,[8,1]]]',
               '[2,9]',
               '[1,[[[9,3],9],[[9,0],[0,7]]]]',
               '[[[5,[7,4]],7],1]','[[[[4,2],2],6],[8,7]]']

bsnails = [Snail(t) for t in big_example]


In [26]:
bsnails[0]

[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]

In [27]:
#Note if you rerun this twice the result will be False
bsnails[0].add(bsnails[1])
print(bsnails[0])
bsnails[0].hw == '[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]' 


[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]


True

In [28]:
bsnails[0].add(bsnails[2])
print(bsnails[0])
bsnails[0].hw == '[[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]'

[[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]


True

In [29]:
####
#   This example initially produced the issue that found the bug in single_boom.  
##  Because when we add t + k in this order, the result was 
#     weird.
####



t = Snail('[2,3]')
u = Snail('[[8,4],3]')
#Examples pre and post where explosions are necessary
k = Snail('[[[[[9,8],1],2],3],4]')
pk = Snail('[[[[0,9],2],3],4]')

print(t)
print(k)
t.add(k)
t,k

[2,3]
[[[[[9,8],1],2],3],4]


([[2,3],[[[5,0],9],4]], [[[[0,9],2],3],4])

In [30]:
vey = Snail('[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]')
vey

[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]

In [31]:
sip = Snail(vey.single_boom())
assert(sip.hw == '[[[[0,7],4],[7,[[8,4],9]]],[1,1]]')
sip

[[[[0,7],4],[7,[[8,4],9]]],[1,1]]

In [32]:
drip = Snail(sip.single_boom())
#assert(drip.hw == '[[[[0,7],4],[15,[0,13]]],[1,1]]')
drip

[[[[0,7],4],[15,0,13]]],[1,1]]

In [33]:
assert(magnitude(Snail('[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]')) == 3488)